In [178]:

import math
import time as tm

import numpy as np

In [179]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [180]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [181]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    def plant_clique():
        for ii in range(clique_size):
            for jj in range(ii + 1, clique_size):
                for kk in range(jj + 1, clique_size):
                    aa = clique_v[ii]
                    # print(a)
                    vec[aa] += 1
                    bb = clique_v[jj]
                    # print(b)
                    vec[bb] += 1
                    cc = clique_v[kk]
                    # print(c)
                    vec[cc] += 1
                    add_all(aa, bb, cc, g)

    g = np.array([np.array([np.array([0 for _ in range(0, num_vertices)]) for _ in range(num_vertices)]) for _ in
                  range(num_vertices)])
    vec = np.array([0 for _ in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    clique_v = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_v


In [182]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [183]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == math.comb(active_count, 3):
        return True
    return False

In [184]:
# remove edge
def remove_edges(g, num_vertices, vec, curr_idx):
    vec[curr_idx] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr_idx, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr_idx, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph
n: number of vertices
p: probability of an edge being included
k: clique size

In [185]:
# driver program

N = 100  # number of vertices
P = 0.5  # probability of an edge being included
K = 10  # clique size

# generate graph
start_generate = tm.time()
res = generate_graph(N, P, K)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertex in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0], res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurrence vector: {V}\n")
print(f"set of clique vertices after removal phase: {clique_vertices}\n")

time_generate = np.round(tm.time() - start_generate, 3)
# print("Time taken to generate graph: ", "seconds")


edge-occurrence vector: [2472 2464 2437 2459 2499 2407 2467 2464 2402 2447 2412 2417 2402 2437
 2464 2444 2420 2421 2390 2493 2395 2493 2451 2459 2475 2425 2420 2463
 2438 2382 2431 2478 2450 2488 2417 2372 2464 2449 2405 2432 2487 2398
 2469 2515 2410 2470 2412 2370 2421 2479 2438 2429 2443 2361 2430 2460
 2402 2461 2470 2373 2371 2488 2441 2475 2446 2472 2423 2401 2405 2388
 2421 2456 2420 2406 2452 2465 2488 2451 2436 2427 2439 2480 2404 2431
 2445 2450 2412 2438 2433 2437 2456 2436 2441 2455 2456 2401 2403 2413
 2411 2438]

set of clique vertices after removal phase: [27  0 58 30 63 96 21 84 76 26]



### Removal Phase

In [186]:
start_removal = tm.time()
itr = 0
removed = []  # keep track of the vertices removed from the original graph to form a clique

# clique = is_clique(G, V, N)

while not is_clique(G, V, N):

    itr += 1

    curr = -1
    idx_sorted = np.argsort(V)

    print(f"i = {itr}\nindex sorted: {idx_sorted}\n")
    # print(f"number of edges associated with each vertex: {V}")

    for idx in range(N):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    # print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, N, V, curr)
    G = A[0]

    clique = is_clique(G, V, N)

print(f"is a clique at iteration #{itr}!!!")
time_remove = np.round(tm.time() - start_removal, 3)

i = 1
index sorted: [53 47 60 35 59 29 69 18 20 41 67 95 56  8 12 96 82 38 68 73  5 44 98 86
 46 10 97 34 11 26 16 72 70 48 17 66 25 79 51 54 83 30 39 88 91 78  2 89
 13 50 99 87 28 80 92 62 52 15 84 64  9 37 85 32 77 22 74 93 71 90 94  3
 23 55 57 27  7 36  1 14 75  6 42 58 45  0 65 63 24 31 49 81 40 76 61 33
 21 19  4 43]

i = 2
index sorted: [53 60 47 29 35 59 69 20 18 12 95 41 96 82 67 56 17 44 97  5 38 86 10 68
  8 73 46 98 34 11 70 48 16 72 26 88 79 39 99 80 54 15 83 78 66 30 50  2
 13 91 52 62 51 87 25 28 32 64 85 84 89 92 22  9 37 77 94 74 71 90  3 93
 55 57 23  1 36 14 42 65 27 63 75  7  6 31 45 58 24  0 49 81 19 61 40 33
 21 76  4 43]

i = 3
index sorted: [60 53 29 47 35 59 69 82 20 18 95 17 44 56 12 41 97 11 96 68  5  8 98 26
 38 86 10 67 46 34 73 48 70 72 88 54 16 83 39 79 78 99 52 80 13 30 87 15
 62 25 51 91 32 92  2 66 37 94 50 64 28 89 22  9 84 77 90 74 85 93  3 57
 71 55 23 36  7  6 14 65 31  1 75 63 42 27 58 81  0 45 24 49 33 19 40  4
 21 61 76 43]

i = 4
index sorted:

In [187]:
print(f"number of iterations in the removal phase: {itr}")

number of iterations in the removal phase: 91


In [188]:
full_vertices = np.arange(N)
included = np.setdiff1d(full_vertices, removed)
assert len(removed) + len(included) == N
print(f"vertices included after the removal phase: {included}")

vertices included after the removal phase: [ 0 21 26 27 58 63 76 84 96]


### Inclusion Phase

In [189]:
def inclusion_phase(target, in_set, g):
    def connected():
        for j in range(len(in_set)):
            for k in range(j + 1, len(in_set)):
                if g[target_idx, in_set[j], in_set[k]] != 1:
                    # print(f" vertex {idx} is not connected to all clique vertices")
                    return False
        return True

    for target_idx in target:
        if connected():
            print(f"add {target_idx} to clique!")
            in_set = np.append(in_set, idx)

    return in_set


In [190]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time() - start_include, 3)

add 30 to clique!


In [191]:
print(f"Set of clique vertices after generation phase: {np.sort(clique_vertices)}\n")
print(f"Set of clique vertices after removal phase: {np.sort(included)}\n")
# print(f"set of vertices removed: {removed}, number of elements = {len(removed)}\n")
print(f"Set of clique vertices after inclusion phase: {np.sort(res)}")

Set of clique vertices after generation phase: [ 0 21 26 27 30 58 63 76 84 96]

Set of clique vertices after removal phase: [ 0 21 26 27 58 63 76 84 96]

Set of clique vertices after inclusion phase: [ 0 21 26 27 58 63 76 84 90 96]
